In [19]:
%load_ext jupyter_black
import os
from glob import glob
from typing import NamedTuple

import pandas as pd
import numpy as np
from numpy.typing import NDArray
from datetime import datetime

import pandas as pd
import json
import numpy as np
import requests
from geopandas import GeoDataFrame
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.interval import IntervalTrigger

# scheduler = BlockingScheduler()

NCEP_DATA = "https://mrms.ncep.noaa.gov/data"


def name_to_datetime(names: pd.Series) -> pd.DatetimeIndex:
    return pd.DatetimeIndex(names.str.replace("_", "T").str.extract(r"(\d*T\d*).json")[0]).rename("validTime")


def read_mrms(*args: str) -> pd.DataFrame:
    url = "/".join([NCEP_DATA, *args]) + "/?C=M;O=D"
    return pd.read_html(url)[0].dropna()


def read_probsevere() -> pd.DataFrame:
    df = read_mrms("ProbSevere", "PROBSEVERE")
    df.index = name_to_datetime(df.Name)
    return (NCEP_DATA + "/ProbSevere/PROBSEVERE/" + df["Name"]).rename("url")


dex = {"probsevere": ("ProbSevere", "PROBSEVERE")}


def extract_html(arg: str):
    arg = arg.upper()
    match arg:
        case "PROBSEVERE":
            return read_probsevere()
        case _:
            raise NotImplementedError


# def transform_to_dataframe():
#     ...
from typing import Mapping

files = extract_html("probsevere")


def to_dataframe(mrms_files: Mapping[pd.Timestamp, str]):
    def generate():
        for vt, url in mrms_files.items():
            for feat in requests.get(url).json()["features"]:
                props = feat["properties"]
                props["validTime"] = vt
                props["geometry"] = feat["geometry"]
                yield props

    ps = pd.DataFrame(generate()).set_index(["validTime", "ID"])

    ps["AVG_BEAM_HGT"] = ps["AVG_BEAM_HGT"].str.replace(r"[A-Za-z]", "", regex=True).apply(pd.eval)

    ps[["MAXRC_EMISS", "MAXRC_ICECF"]] = (
        ps[["MAXRC_EMISS", "MAXRC_ICECF"]]
        .stack()
        .str.extract(r"(?:\()([a-z]*)(?:\))")
        .replace({"weak": 1, "moderate": 2, "strong": 3})
        .fillna(0)
        .unstack(-1)
        .droplevel(0, axis=1)
    )
    # ps[[ps.columns != "geometry"]] = ps[ps.columns != "geometry"].astype(np.float32)
    ps.loc[:, ps.columns != "geometry"] = ps.loc[:, ps.columns != "geometry"].astype(np.float32)
    return ps


df = to_dataframe(files.tail())
df

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
validTime           ID                                                  
2022-05-28 14:08:36 133672  3202.0  2329.0   -4.0   8.600000     49.0   
                    133697  3056.0  2257.0   -4.0  16.299999     39.0   
                    133866  3059.0  2427.0   -4.0   4.800000     22.0   
                    133947   931.0     0.0   -4.0  25.000000    257.0   
                    133997  2211.0  1778.0   -4.0   7.500000     26.0   
...                            ...     ...    ...        ...      ...   
2022-05-28 14:00:39 134535  1330.0   152.0 -551.0  39.200001    188.0   
                    134536  2930.0  2130.0   -4.0   5.800000     26.0   
                    134537  2646.0  2069.0   -4.0  14.700000     21.0   
                    134538  2820.0  1966.0   -4.0  15.600000     21.0   
                    134539  3095.0  2062.0   -4.0  15.300000     19.0   

                            MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
validTime           ID                                                         
2022-05-28 14:08:36 133672          31.799999  0.31         0.71         7.0   
                    133697          29.200001  0.03         0.35         0.0   
                    133866          29.100000  0.28         0.61         3.0   
                    133947          21.200001  0.06         0.48         1.0   
                    133997          14.000000  0.11         1.28         0.0   
...                                       ...   ...          ...         ...   
2022-05-28 14:00:39 134535          40.599998  0.00         0.40         0.0   
                    134536          31.700001  0.00         0.27         0.0   
                    134537          15.800000  0.00         0.48         0.0   
                    134538          15.000000  0.06         1.16         0.0   
                    134539          12.700000  0.00         0.51         0.0   

                            FLASH_DENSITY  ...  WETBULB_0C_HGT  PWAT  \
validTime           ID                     ...                         
2022-05-28 14:08:36 133672           0.20  ...            12.3   2.1   
                    133697           0.03  ...            13.2   2.1   
                    133866           0.32  ...            12.6   2.0   
                    133947           0.20  ...            12.3   1.4   
                    133997           0.05  ...            10.5   1.4   
...                                   ...  ...             ...   ...   
2022-05-28 14:00:39 134535           0.00  ...            11.8   1.2   
                    134536           0.03  ...            13.1   2.2   
                    134537           0.00  ...            13.0   1.9   
                    134538           0.00  ...            12.2   1.9   
                    134539           0.00  ...            12.3   1.7   

                            CAPE_M10M30  LJA   SIZE  AVG_BEAM_HGT  \
validTime           ID                                              
2022-05-28 14:08:36 133672        651.0  0.0  101.0      3.278688   
                    133697        591.0  0.0   73.0      3.281818   
                    133866        604.0  0.0   98.0      3.279026   
                    133947        271.0  0.0  211.0      3.289474   
                    133997        574.0  0.0  255.0      3.274194   
...                                 ...  ...    ...           ...   
2022-05-28 14:00:39 134535        240.0  0.0   47.0      3.269939   
                    134536        574.0  0.0   56.0      3.280269   
                    134537        607.0  0.0   63.0      3.258065   
                    134538        596.0  0.0   86.0      3.278846   
                    134539        653.0  0.0   77.0      3.333333   

                            MOTION_EAST  MOTION_SOUTH   PS  \
validTime           ID                                       
2022-05-28 14:08:36 133672        7.619        -6.162  1.0   
                

In [5]:
from typing import List
class ProbSevereFeature(pydantic.BaseModel):...

class ProbSevere(pydantic.BaseModel):
    features:List[ProbSevereFeature]

In [46]:
import pydantic

ModuleNotFoundError: No module named 'pydantic'

In [43]:
df.to_parquet("test.parquet", engine="pyarrow")

In [45]:
pd.read_parquet("test.parquet", engine="pyarrow")

MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime           ID                                           
2022-05-28 14:08:36 133672   3202   2329    -4     8.6      49   
                    133697   3056   2257    -4    16.3      39   
                    133866   3059   2427    -4     4.8      22   
                    133947    931      0    -4    25.0     257   
                    133997   2211   1778    -4     7.5      26   
...                           ...    ...   ...     ...     ...   
2022-05-28 14:00:39 134535   1330    152  -551    39.2     188   
                    134536   2930   2130    -4     5.8      26   
                    134537   2646   2069    -4    14.7      21   
                    134538   2820   1966    -4    15.6      21   
                    134539   3095   2062    -4    15.3      19   

                           MEANWIND_1-3kmAGL  MESH VIL_DENSITY FLASH_RATE  \
validTime           ID                                                      
2022-05-28 14:08:36 133672              31.8  0.31        0.71          7   
                    133697              29.2  0.03        0.35          0   
                    133866              29.1  0.28        0.61          3   
                    133947              21.2  0.06        0.48          1   
                    133997              14.0  0.11        1.28          0   
...                                      ...   ...         ...        ...   
2022-05-28 14:00:39 134535              40.6  0.00        0.40          0   
                    134536              31.7  0.00        0.27          0   
                    134537              15.8  0.00        0.48          0   
                    134538              15.0  0.06        1.16          0   
                    134539              12.7  0.00        0.51          0   

                           FLASH_DENSITY  ... WETBULB_0C_HGT PWAT CAPE_M10M30  \
validTime           ID                    ...                                   
2022-05-28 14:08:36 133672          0.20  ...           12.3  2.1         651   
                    133697          0.03  ...           13.2  2.1         591   
                    133866          0.32  ...           12.6  2.0         604   
                    133947          0.20  ...           12.3  1.4         271   
                    133997          0.05  ...           10.5  1.4         574   
...                                  ...  ...            ...  ...         ...   
2022-05-28 14:00:39 134535          0.00  ...           11.8  1.2         240   
                    134536          0.03  ...           13.1  2.2         574   
                    134537          0.00  ...           13.0  1.9         607   
                    134538          0.00  ...           12.2  1.9         596   
                    134539          0.00  ...           12.3  1.7         653   

                            LJA  SIZE AVG_BEAM_HGT MOTION_EAST MOTION_SOUTH  \
validTime           ID                                                        
2022-05-28 14:08:36 133672  0.0   101     3.278689       7.619       -6.162   
                    133697  0.0    73     3.281818       6.693       -7.198   
                    133866  0.0    98     3.279026        8.63       -3.626   
                    133947  0.0   211     3.289474       6.064       -7.198   
                    133997  0.0   255     3.274194       3.196       -3.516   
...                         ...   ...          ...         ...          ...   
2022-05-28 14:00:39 134535  0.0    47     3.269939      10.469       -0.942   
                    134536  0.0    56     3.280269       7.641       -3.603   
                    134537  0.0    63     3.258065      -0.283       -2.271   
                    134538  0.0    86     3.278846       2.092       -0.247   
                    134539  0.0    77     3.333333         4.1          0.5   

                           PS  \
validTime           ID          
2022-05-28 14:08:3

In [17]:
from datetime import datetime

import pandas as pd
import json
import numpy as np
import requests
from geopandas import GeoDataFrame
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.interval import IntervalTrigger

scheduler = BlockingScheduler()


def name_to_datetime(names: pd.Series) -> pd.DatetimeIndex:
    return pd.DatetimeIndex(names.str.replace("_", "T").str.extract(r"(\d*T\d*).json")[0]).rename("validTime")


def read_mrms(*args: str) -> pd.DataFrame:
    url = "/".join(["https://mrms.ncep.noaa.gov/data", *args]) + "/?C=M;O=D"
    return pd.read_html(url)[0].dropna()


def read_probsevere() -> pd.DataFrame:
    df = read_mrms("ProbSevere", "PROBSEVERE")
    df.index = name_to_datetime(df.Name)
    return ("https://mrms.ncep.noaa.gov/data/ProbSevere/PROBSEVERE/" + df["Name"]).rename("url")


def to_dataframe(urls: pd.Series, source="URL") -> pd.DataFrame:
    def generate():
        for vt, path in urls.items():
            if source == "URL":
                feat = requests.get(path).json()

            if source == "PATH":
                with open(path, mode="r", encoding="utf-8") as f:
                    feat = json.load(f)
            df = GeoDataFrame.from_features(feat["features"])
            df["validTime"] = vt
            yield df

    # concat the iterable files
    ps = pd.concat(generate(), ignore_index=True)
    ps = pd.concat([ps, ps["geometry"].bounds], axis=1)
    # convert string values to
    ps["AVG_BEAM_HGT"] = ps["AVG_BEAM_HGT"].str.replace(r"[A-Za-z]", "", regex=True).apply(pd.eval)

    ps[["MAXRC_EMISS", "MAXRC_ICECF"]] = (
        ps[["MAXRC_EMISS", "MAXRC_ICECF"]]
        .stack()
        .str.extract(r"(?:\()([a-z]*)(?:\))")
        .replace({"weak": 1, "moderate": 2, "strong": 3})
        .fillna(0)
        .unstack(-1)
        .droplevel(0, axis=1)
    )

    return (
        ps.set_index(["validTime", "ID", "minx", "miny", "maxx", "maxy"]).drop("geometry", axis=1).astype(np.float32)
    )


@scheduler.scheduled_job(IntervalTrigger(minutes=10))
def on_interval():
    print(f"begining interval scheduled task at {datetime.now()}\n")
    # the previously downloaded dataset
    # this should be updated to save the data into files based on the utc date
    df = pd.read_parquet("data/PROBSEVERE.parquet", engine="pyarrow")
    # READ FROM MRMS DATASET
    urls = read_probsevere()
    # LOCATE FILES THAT HAVE NOT BEEN DOWNLOADED
    data_to_get = urls.loc[~urls.index.unique("validTime").isin(df.index.unique("validTime"))]
    if not data_to_get.any():
        print("NO NEW FILES FOUND")
        return
    getting = data_to_get.tail(20)

    print(f"getting {len(getting)} files valid for", ", ".join(getting.index.astype(str).tolist()), "\n\n")
    # DOWNLOAD NEWFILES
    new_files = to_dataframe(getting, source="URL")
    # JOIN THE FILES
    pd.concat([df, new_files]).to_parquet("data/PROBSEVERE.parquet", engine="pyarrow")
    print("FILE SAVED")


if __name__ == "__main__":
    on_interval()
    scheduler.start()

begining interval scheduled task at 2022-05-29 11:24:35.621737

getting 20 files valid for 2022-05-28 14:38:40, 2022-05-28 14:36:39, 2022-05-28 14:34:42, 2022-05-28 14:32:38, 2022-05-28 14:30:38, 2022-05-28 14:28:40, 2022-05-28 14:26:42, 2022-05-28 14:24:39, 2022-05-28 14:22:40, 2022-05-28 14:20:41, 2022-05-28 14:18:42, 2022-05-28 14:16:38, 2022-05-28 14:14:39, 2022-05-28 14:12:40, 2022-05-28 14:10:37, 2022-05-28 14:08:36, 2022-05-28 14:06:38, 2022-05-28 14:04:39, 2022-05-28 14:02:41, 2022-05-28 14:00:39 




KeyboardInterrupt: 

In [15]:
# read_mrms("ProbSevere", "PROBSEVERE")
# read_probsevere()
extract_html("probsevere")

NameError: name 'df' is not defined